# Функции для автоматизированной системы разложения элементов группы Липшица в произведение векторов в алгебрах Клиффорда


Ссылка на веб-приложение: https://lgroup.streamlit.app/

Ссылка на гит: https://github.com/katyafilimoshina/lgd

Содержание блокнота:

* [Установка и импорт библиотек и пакетов](#1)
* [Определение алгебры Клиффорда](#2)
* [Проверка принадлежности элемента группе Липшица](#3)
* [Разложение элемента группы Липшица в произведение обратимых векторов](#4)
  * [Разложение в случае $n=1$](#4.1)
  * [Разложение в случае $n=2$](#4.2)
  * [Разложение в случае $n=3$](#4.3)
  * [Общая функция для разложения](#4.4)
  * [Проверка на примерах](#4.5)
* [Визуализация результатов в случае $C\!\ell_{2,0}$](#5)


## Установка и импорт библиотек и пакетов <a class="anchor" id="1"></a>

In [4]:
# # можно раскомментировать код ниже, если нужно установить пакет clifford
# !pip install clifford

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 10.1 MB/s eta 0:00:00


In [5]:
import clifford as cf
from math import sqrt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Определение алгебры Клиффорда <a class="anchor" id="2"></a>

In [6]:
p = 2 
q = 1

layout, blades = cf.Cl(p,q) # задаёт алгебру Клиффорда Cl(p,q) размерности p+q
locals().update(blades)

values = blades.values()
elements = []
for value in values:
      elements.append(value)

generators = []
for value in elements:
      if value(1) != 0:
          generators.append(value)

blades_dict = dict((i, list(blades.keys())[i]) for i in range(len(blades)))

## Проверка принадлежности элемента группе Липшица <a class="anchor" id="3"></a>

In [7]:
def if_invertible_LG(element):
  '''
  Проверяет обратимость элемента группы Липшица

  :element: элемент алгебры Клиффорда
  :return: True/False, если принадлежит/не принадлежит
  '''
  norm_value = ~element * element
  if ((norm_value(0) == norm_value) & (norm_value != 0)):
    return True
  else:
    return False

In [8]:
def inverse_LG(element):
  '''
  Находит обратный элемент для элемента группы Липшица

  :element: элемент группы Липшица
  :return: обратный элемент
  '''
  norm_value = ~element * element
  return ~element / norm_value

In [9]:
def if_even(element):
  '''
  Проверяет чётность элемента

  :element: элемент алгебры Клиффорда
  :return: True/False, если является/не является чётным
  '''
  if element.even - element == 0:
    return True
  else:
    return False

In [10]:
def if_odd(element):
  '''
  Проверяет нечётность элемента

  :element: элемент алгебры Клиффорда
  :return: True/False, если является/не является нечётным
  '''
  if element.odd - element == 0:
    return True
  else:
    return False

In [11]:
def if_preserves_grade1(element):
  '''
  Проверяет, сохраняет ли элемент подпространство ранга 1 при присоединённом действии

  :element: элемент алгебры Клиффорда
  :return: True/False, если сохраняет/не сохраняет
  '''
  for generator in generators:
    ad = element * generator * ~element
    if ad(1) - ad != 0:
      return False
  return True

In [12]:
def if_in_LG(element):
  '''
  Проверяет, принадлежит ли элемент группе Липшица

  :element: элемент алгебры Клиффорда
  :return: True, если принадлежит; если не принадлежит, причина
  '''
  invertible, even, odd, preserves_grade1  = if_invertible_LG(element), if_even(element), if_odd(element), if_preserves_grade1(element)

  if invertible & (even | odd) & preserves_grade1:
    return True
  
  if invertible == False:
    return "Элемент НЕ принадлежит группе Липшица: значение функции нормы элемента $$\psi(T)=\widetilde{T}T$$ \
        не принадлежит подпространству ранга 0 или равно нулю."

  if (even | odd) == False:
    return "Элемент НЕ принадлежит группе Липшица: он не является ни чётным, ни нечётным."

  if preserves_grade1 == False:
    return "Элемент НЕ принадлежит группе Липшица: он не сохраняет подпространство ранга 1 при ad."

## Разложение элемента группы Липшица в произведение обратимых векторов <a class="anchor" id="4"></a>

### Разложение в случае $n=1$ <a class="anchor" id="4.1"></a>

In [13]:
def factorization_n1(element):
  '''
  Раскладывает элемент группы Липшица в произведение векторов в случае n = 1

  :element: элемент группы Липшица
  :return: list() с множителями в разложении
  '''
  eta_11 = e1 ** 2

  if if_even(element) == True:
    return [(element(0) * eta_11 * e1), e1]
  else:
    return [element]

### Разложение в случае $n=2$ <a class="anchor" id="4.2"></a>


In [14]:
def factorization_n2(element):
  '''
  Раскладывает элемент группы Липшица в произведение векторов в случае n = 2

  :element: элемент группы Липшица
  :return: list() с множителями в разложении
  '''
  eta_11 = e1 ** 2

  u_dict = dict((i, element.value[i]) for i in range(len(element.value)))
  element_dict = {v: u_dict[k] for k, v in blades_dict.items()}

  if if_even(element) == True:
    return [e1, element_dict[''] * eta_11 * e1 + element_dict['e12'] * e2]
  else:
    return [element]

### Разложение в случае $n=3$ <a class="anchor" id="4.3"></a>

In [15]:
def factorization_n3(element):
  '''
  Раскладывает элемент группы Липшица в произведение векторов в случае n = 3

  :element: элемент группы Липшица
  :return: list() с множителями в разложении
  '''
  eta_11, eta_22, eta_33 = e1 ** 2, e2 ** 2, e3 ** 2

  u_dict = dict((i, element.value[i]) for i in range(len(element.value)))
  element_dict = {v: u_dict[k] for k, v in blades_dict.items()}

  if (element(1) - element) == 0:
    return [element]

  elif if_even(element) == True:
    if element_dict['e12'] == 0:
      return [e3, -element_dict['e13'] * e1 - element_dict['e23'] * e2 + element_dict[''] * eta_33 * e3]

    elif element_dict['e13'] == 0:
      return [e2, -element_dict['e12'] * e1 + element_dict[''] * eta_22 * e2 + element_dict['e23'] * e3]

    elif element_dict['e23'] == 0:
      return [e1, element_dict[''] * eta_11 * e1 + element_dict['e12'] * e2 + element_dict['e13'] * e3]

    elif (element_dict['e13']) ** 2 * eta_11 + (element_dict['e23']) ** 2 * eta_22 != 0:
      lambda_coef = (element_dict['e13']) ** 2 * eta_11 + (element_dict['e23']) ** 2 * eta_22
      return [(1 / lambda_coef) * (element_dict['e13'] * e1 + element_dict['e23'] * e2), (element_dict[''] * element_dict['e13'] - element_dict['e12'] * element_dict['e23'] * eta_22) * e1 + (element_dict[''] * element_dict['e23'] + element_dict['e12'] * element_dict['e13'] * eta_11) * e2 + lambda_coef * e3]

    elif (element_dict['e12']) ** 2 * eta_11 + (element_dict['e23']) ** 2 * eta_33 != 0:
      lambda_coef = (element_dict['e12']) ** 2 * eta_11 + (element_dict['e23']) ** 2 * eta_33
      return [(1 / lambda_coef) * (- element_dict['e12'] * e1 + element_dict['e23'] * e3), (- element_dict[''] * element_dict['e12'] - element_dict['e13'] * element_dict['e23'] * eta_33) * e1 - lambda_coef * e2 + (element_dict[''] * element_dict['e23'] - element_dict['e12'] * element_dict['e13'] * eta_11) * e3]

    elif (element_dict['e12']) ** 2 * eta_22 + (element_dict['e13']) ** 2 * eta_33 != 0:
      lambda_coef = (element_dict['e12']) ** 2 * eta_22 + (element_dict['e13']) ** 2 * eta_33
      return [(1 / lambda_coef) * (element_dict['e12'] * e2 + element_dict['e13'] * e3), - lambda_coef * e1 + (element_dict[''] * element_dict['e12'] - element_dict['e13'] * element_dict['e23'] * eta_33) * e2 + (element_dict[''] * element_dict['e13'] + element_dict['e12'] * element_dict['e23'] * eta_22) * e3]

    else:
      return "Упс, что-то пошло не так... Но мы уже чиним это!"

  else:
    factors = [e1]
    factors.extend(factorization_n3(eta_11 * e1 * element))
    return factors

### Общая функция для разложения <a class="anchor" id="4.4"></a>

In [16]:
def factorization(element):
  '''
  Главная функция разложения элементов группы Липшица в произведение векторов

  :element: элемент алгебры Клиффорда
  :return: list() с множителями в разложении
  '''
  element = element + e1 - e1
  in_LG = if_in_LG(element)

  if in_LG == True:
    if p + q == 1:
      return factorization_n1(element)
    if p + q == 2:
      return factorization_n2(element)
    if p + q == 3:
      return factorization_n3(element)
  
  else:
    return in_LG

### Проверка на примерах <a class="anchor" id="4.5"></a>


In [17]:
factorization(1 + 2*e2)

'Элемент НЕ принадлежит группе Липшица: значение функции нормы элемента $$\\psi(T)=\\widetilde{T}T$$         не принадлежит подпространству ранга 0 или равно нулю.'

In [18]:
factorization(2)

[(1^e3), -(2^e3)]

In [19]:
factorization(2 + e13)

[(1^e3), -(1^e1) - (2^e3)]

In [20]:
factorization(1 + 7*e13 - 4*e23)

[(1^e3), -(7^e1) + (4^e2) - (1^e3)]

In [21]:
factorization(1 + 7*e3 - 4*e123)

'Элемент НЕ принадлежит группе Липшица: значение функции нормы элемента $$\\psi(T)=\\widetilde{T}T$$         не принадлежит подпространству ранга 0 или равно нулю.'

In [22]:
factorization(1 + e123)

'Элемент НЕ принадлежит группе Липшица: значение функции нормы элемента $$\\psi(T)=\\widetilde{T}T$$         не принадлежит подпространству ранга 0 или равно нулю.'

In [23]:
factorization(e12 + e13 + e23)

[(0.5^e1) + (0.5^e2), -(1^e1) + (1^e2) + (2^e3)]

In [24]:
factorization(e1 + e2 - e3 - e123)

'Элемент НЕ принадлежит группе Липшица: значение функции нормы элемента $$\\psi(T)=\\widetilde{T}T$$         не принадлежит подпространству ранга 0 или равно нулю.'

## Визуализация результатов в случае $C\!\ell_{2,0}$ <a class="anchor" id="5"></a>

In [25]:
def beautiful_print_vector(vector):
  '''
  Выводит вектор в удобном формате для красивой печати

  :vector: вектор
  :return: вектор в формате latex
  '''
  return sum([x * y for x, y in zip(list(vector.value)[1:p+q+1], list(o3d.mv()))])

In [26]:
def find_max_min_coordinates(vectors):
  '''
  Находит максимальную и минимальную координату среди всех координат массива векторов

  :vector: list() с векторами
  :return: [минимальная координата, максимальная координата]
  '''
  all_coordinates = []
  for vector in vectors:
    all_coordinates.extend(list(dict((i, vector.value[i]) for i in range(len(vector.value))).values())[1:3])
  min_coordinate, max_coordinate = min(all_coordinates), max(all_coordinates)
  return [min_coordinate, max_coordinate]

In [27]:
def get_vector_coordinates(vector):
  '''
  Находит координаты вектора

  :vector: вектор (x1,x2)
  :return: [0, x1], [0, x2]
  '''
  end_coordinates = list(dict((i, vector.value[i]) for i in range(len(vector.value))).values())[1:3]
  return [[0, end_coordinates[0]], [0, end_coordinates[1]]]

In [28]:
def find_all_reflections(vectors, start):
  '''
  Находит все отражения для заданного элемента группы Липшица

  :vectors: list() с векторами
  :start: вектор, на который действуем
  :return: list() со всеми отражениями вектора start, включая start
  '''
  all_reflections = [start]
  applied_to = start
  vectors = list(reversed(vectors))

  for i in range(len(vectors)):
    applied_to = vectors[i].gradeInvol() * applied_to * vectors[i].inv()
    all_reflections.append(applied_to)

  return all_reflections

In [29]:
def figure_reflections(vectors, start):
  '''
  Строит анимированный график отражений

  :vectors: list() с векторами
  :start: вектор, на который действуем
  :return: анимированный график отражений
  '''

  all_reflections = find_all_reflections(vectors, start)
  all_coordinates = []
  for reflection in all_reflections:
    all_coordinates.append(get_vector_coordinates(reflection))
  
  fig_frames = []
  for i in range(len(all_coordinates)):
    fig_frames.append(go.Frame(data=[go.Scatter(x=all_coordinates[i][0], y=all_coordinates[i][1], marker= dict(size=10,symbol= "arrow-bar-up", angleref="previous"))]))
  
  if len(fig_frames) != 2:
    plot_title = f"Композиция {len(fig_frames)-1} отражений"
  else:
    plot_title = "Отражение"
  
  data_to_show = [go.Scatter(x=all_coordinates[0][0], y=all_coordinates[0][1], marker= dict(size=10,symbol= "arrow-bar-up", angleref="previous"), line=dict(width=3, color="#ff6f69"), name="Отражаемый вектор")]

  vectors_coordinates = []
  hyper_coordinates = []
  for vector in vectors:
    vector_c = get_vector_coordinates(vector)
    vectors_coordinates.append(vector_c)
    if vector_c[0][1] == 0:
      hyper_coordinates.append([[-100, 100], [0, 0]])
    elif vector_c[1][1] == 0:
      hyper_coordinates.append([[0, 0], [-100, 100]])
    else:
      hyper_coordinates.append([[(-vector_c[1][1] * 100)/vector_c[0][1], (vector_c[1][1] * 100)/vector_c[0][1]], [100, -100]])

  
  colors = ['#32a852', '#3261a8']

  for i in range(len(vectors_coordinates)):
    data_to_show.append(go.Scatter(x=vectors_coordinates[i][0], y=vectors_coordinates[i][1], marker = dict(size=8,symbol= "arrow-bar-up", angleref="previous"), line=dict(color=colors[i], width=1), name=f"Вектор"))
    data_to_show.append(go.Scatter(x=hyper_coordinates[i][0], y=hyper_coordinates[i][1], line=dict(color=colors[i], width=1), marker=dict(symbol="asterisk"), name=f"Гиперплоскость"))


  fig = go.Figure(
    data=data_to_show,
    layout=go.Layout(
        xaxis=dict(range=[find_max_min_coordinates(all_reflections)[0] - 1, find_max_min_coordinates(all_reflections)[1] + 1]),
        yaxis=dict(range=[find_max_min_coordinates(all_reflections)[0] - 1, find_max_min_coordinates(all_reflections)[1] + 1]),
        autosize=False,
        width=700,
        height=600,
        title=plot_title,
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Показать",
                          method="animate",
                          args=[None, {"frame": {"duration": 2500},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 0}}]),
                     {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Пауза",
                "method": "animate"
            }])]
    ),
    frames=fig_frames
)
   
  fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

  return fig

In [30]:
def figure_rotation(element, start):
  '''
  Строит график поворота

  :element: элемент группы Липшица
  :start: вектор, на который действуем
  :return: график поворота
  '''

  start_coordinates = get_vector_coordinates(start)

  end = element.gradeInvol() * start * element.inv()
  end_coordinates = get_vector_coordinates(end)

  fig = go.Figure(
    data=[go.Scatter(x=start_coordinates[0], y=start_coordinates[1], marker= dict(size=10,symbol= "arrow-bar-up", angleref="previous"), line=dict(width=3, color="#ff6f69"), name="Исходный вектор"), 
          go.Scatter(x=end_coordinates[0], y=end_coordinates[1], marker= dict(size=10,symbol= "arrow-bar-up", angleref="previous"), line=dict(width=3, color='#2ab7ca'), name="Результат поворота")], 
    layout=go.Layout(
        xaxis=dict(range=[find_max_min_coordinates([start, end])[0] - 2, find_max_min_coordinates([start, end])[1] + 2]),
        yaxis=dict(range=[find_max_min_coordinates([start, end])[0] - 2, find_max_min_coordinates([start, end])[1] + 2]),
        autosize=False,
        width=600,
        height=600,
        title="Поворот"))
  
  return fig

Проверка на примере

Рассматриваем элемент Группы Липшица $T= e +2e_{12}=e_1(e_1+2e_2)$ в случае $C\!\ell_{2,0}$. Действуем скрученным присоединённым действием на вектор $e_1-e_2$.


In [31]:
figure_reflections(factorization(1 + 2*e12), e1 - e2)

In [32]:
figure_rotation(1 + 2*e12, e1 - e2)

Результаты поворота и композиции отражений совпали.